In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import norm

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.cluster as cluster

In [2]:
data = pd.read_csv('../credit_card_kaggle/creditcard.csv')
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data_cols = list(data.columns[ data.columns != 'Class' ])
label_cols = ['Class']

In [5]:
minmaxscaler=MinMaxScaler()
minmaxscaler.fit(data[data_cols])
data[data_cols] = minmaxscaler.transform(data[data_cols])

In [6]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000000,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0
1,0.000000,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0
2,0.000006,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0
3,0.000006,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0
4,0.000012,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0


In [7]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

In [8]:
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        # https://pytorch.org/docs/stable/nn.html.
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

In [9]:
# Verify the generator block function
def test_gen_block(in_features, out_features, num_test=1000):
    block = get_generator_block(in_features, out_features)

    # Check the three parts
    assert len(block) == 3
    assert type(block[0]) == nn.Linear
    assert type(block[1]) == nn.BatchNorm1d
    assert type(block[2]) == nn.ReLU
    
    # Check the output shape
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features)
    assert test_output.std() > 0.55
    assert test_output.std() < 0.65

test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

Success!


In [10]:
#  Generator
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
          (MNIST images are 28 x 28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=10, im_dim=30, hidden_dim=20):
        super(Generator, self).__init__()
        # Build the neural network
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            # There is a dropdown with hints if you need them!
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

In [11]:
# Verify the generator class
def test_generator(z_dim, im_dim, hidden_dim, num_test=10000):
    gen = Generator(z_dim, im_dim, hidden_dim).get_gen()
    
    # Check there are six modules in the sequential part
    assert len(gen) == 6
    assert str(gen.__getitem__(4)).replace(' ', '') == f'Linear(in_features={hidden_dim * 8},out_features={im_dim},bias=True)'
    assert str(gen.__getitem__(5)).replace(' ', '') == 'Sigmoid()'
    test_input = torch.randn(num_test, z_dim)
    test_output = gen(test_input)

    # Check that the output shape is correct
    assert tuple(test_output.shape) == (num_test, im_dim)
    assert test_output.max() < 1, "Make sure to use a sigmoid"
    assert test_output.min() > 0, "Make sure to use a sigmoid"
    assert test_output.std() > 0.05, "Don't use batchnorm here"
    assert test_output.std() < 0.15, "Don't use batchnorm here"

test_generator(5, 10, 20)
test_generator(20, 8, 24)
print("Success!")

Success!


In [12]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    # NOTE: To use this on GPU with device='cuda', make sure to pass the device 
    # argument to the function you use to generate the noise.
    return torch.randn(n_samples,z_dim,device=device)


In [13]:
# Verify the noise vector function
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = get_noise(n_samples, z_dim, device)
    # Make sure a normal distribution was used
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)

test_get_noise(1000, 100, 'cpu')
if torch.cuda.is_available():
    test_get_noise(1000, 32, 'cuda')
print("Success!")

Success!


In [14]:
# get_discriminator_block
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a discriminator neural network layer, with a linear transformation 
          followed by an nn.LeakyReLU activation with negative slope of 0.2 
          (https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html)
    '''
    return nn.Sequential(
         nn.Linear(input_dim, output_dim), #Layer 1
         nn.LeakyReLU(0.2, inplace=True)
    )

In [15]:
#  Verify the discriminator block function
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)

    # Check there are two parts
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)

    # Check that the shape is right
    assert tuple(test_output.shape) == (num_test, out_features)
    
    # Check that the LeakyReLU slope is about 0.2
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5
    
    assert str(block.__getitem__(0)).replace(' ', '') == f'Linear(in_features={in_features},out_features={out_features},bias=True)'        
    assert str(block.__getitem__(1)).replace(' ', '').replace(',inplace=True', '') == 'LeakyReLU(negative_slope=0.2)'


test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Success!


In [16]:
# GRADED FUNCTION: Discriminator
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28x28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=30, hidden_dim=10):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            # Hint: You want to transform the final output into a single value,
            #       so add one more linear map.
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        return self.disc(image.float())
    
    # Needed for grading
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc

In [17]:
# Verify the discriminator class
def test_discriminator(z_dim, hidden_dim, num_test=100):
    
    disc = Discriminator(z_dim, hidden_dim).get_disc()

    # Check there are three parts
    assert len(disc) == 4
    assert type(disc.__getitem__(3)) == nn.Linear

    # Check the linear layer is correct
    test_input = torch.randn(num_test, z_dim)
    test_output = disc(test_input)
    assert tuple(test_output.shape) == (num_test, 1)

test_discriminator(5, 10)
test_discriminator(20, 8)
print("Success!")

Success!


In [18]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 10
z_dim = 10
hidden_dim=20
display_step = 500
batch_size = 16
lr = 0.00001
device = 'cpu'

In [19]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

In [20]:
# get_disc_loss
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''
    #     These are the steps you will need to complete:
    #       1) Create noise vectors and generate a batch (num_images) of fake images. 
    #            Make sure to pass the device argument to the noise.
    #       2) Get the discriminator's prediction of the fake image 
    #            and calculate the loss. Don't forget to detach the generator!
    #            (Remember the loss function you set earlier -- criterion. You need a 
    #            'ground truth' tensor in order to calculate the loss. 
    #            For example, a ground truth tensor for a fake image is all zeros.)
    #       3) Get the discriminator's prediction of the real image and calculate the loss.
    #       4) Calculate the discriminator's loss by averaging the real and fake loss
    #            and set it to disc_loss.

    fake_noise = get_noise(num_images, z_dim, device=device)
    fake = gen(fake_noise)
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss

In [21]:
features= data[data_cols]
type(features)

pandas.core.frame.DataFrame

In [22]:
targets = data[['Class']]
type(targets)

pandas.core.frame.DataFrame

In [23]:
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(torch.tensor(features.values,dtype=torch.float64), torch.tensor(targets.values,dtype=torch.float64))
train_loader = DataLoader(train, batch_size=16, shuffle=True)



In [24]:
dataloader=DataLoader(train, batch_size=16, shuffle=True)

In [25]:
def test_disc_reasonable(num_images=10):
    z_dim = 64
    gen = torch.zeros_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    real = torch.ones(num_images, 1)
    disc_loss = get_disc_loss(gen, disc, criterion, real, num_images, z_dim, 'cpu')
    assert tuple(disc_loss.shape) == (num_images, z_dim)
    assert torch.all(torch.abs(disc_loss - 0.5) < 1e-5)

    gen = torch.ones_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    real = torch.zeros(num_images, 1)
    assert torch.all(torch.abs(get_disc_loss(gen, disc, criterion, real, num_images, z_dim, 'cpu')) < 1e-5)

def test_disc_loss(max_tests = 10):
    z_dim = 64
    gen = Generator(z_dim).to(device)
    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
    disc = Discriminator().to(device) 
    disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
    num_steps = 0
    for real in dataloader:
        real=real[0]
        cur_batch_size = len(real)
        real = real.view(cur_batch_size, -1).to(device)
#         real=torch.tensor(real).view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradient before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)
        assert (disc_loss - 0.68).abs() < 0.05

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Check that they detached correctly
        assert gen.gen[0][0].weight.grad is None

        # Update optimizer
        old_weight = disc.disc[0][0].weight.data.clone()
        disc_opt.step()
        new_weight = disc.disc[0][0].weight.data
        
        # Check that some discriminator weights changed
        assert not torch.all(torch.eq(old_weight, new_weight))
        num_steps += 1
        if num_steps >= max_tests:
            break

test_disc_reasonable()
test_disc_loss()
print("Success!")

Success!


In [26]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return the loss of the generator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        gen_loss: a torch scalar loss value for the current batch
    '''
    #     These are the steps you will need to complete:
    #       1) Create noise vectors and generate a batch of fake images. 
    #           Remember to pass the device argument to the get_noise function.
    #       2) Get the discriminator's prediction of the fake image.
    #       3) Calculate the generator's loss. Remember the generator wants
    #          the discriminator to think that its fake images are real
    #     *Important*: You should NOT write your own loss function here - use criterion(pred, true)!

    fake_noise = get_noise(num_images, z_dim, device=device)
    fake = gen(fake_noise)
    disc_fake_pred = disc(fake)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss

In [27]:
def test_gen_reasonable(num_images=10):
    z_dim = 64
    gen = torch.zeros_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    gen_loss_tensor = get_gen_loss(gen, disc, criterion, num_images, z_dim, 'cpu')
    assert torch.all(torch.abs(gen_loss_tensor) < 1e-5)
    #Verify shape. Related to gen_noise parametrization
    assert tuple(gen_loss_tensor.shape) == (num_images, z_dim)

    gen = torch.ones_like
    disc = nn.Identity()
    criterion = torch.mul # Multiply
    real = torch.zeros(num_images, 1)
    gen_loss_tensor = get_gen_loss(gen, disc, criterion, num_images, z_dim, 'cpu')
    assert torch.all(torch.abs(gen_loss_tensor - 1) < 1e-5)
    #Verify shape. Related to gen_noise parametrization
    assert tuple(gen_loss_tensor.shape) == (num_images, z_dim)
    

def test_gen_loss(num_images):
    z_dim = 64
    gen = Generator(z_dim).to(device)
    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
    disc = Discriminator().to(device) 
    disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
    
    gen_loss = get_gen_loss(gen, disc, criterion, num_images, z_dim, device)
    
    # Check that the loss is reasonable
    assert (gen_loss - 0.7).abs() < 0.1
    gen_loss.backward()
    old_weight = gen.gen[0][0].weight.clone()
    gen_opt.step()
    new_weight = gen.gen[0][0].weight
    assert not torch.all(torch.eq(old_weight, new_weight))


test_gen_reasonable(10)
test_gen_loss(18)
print("Success!")

Success!


In [28]:
from numpy import dot
from numpy.linalg import norm

In [29]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True # Whether the generator should be tested
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        ### Update generator ###
        #     Hint: This code will look a lot like the discriminator updates!
        #     These are the steps you will need to complete:
        #       1) Zero out the gradients.
        #       2) Calculate the generator loss, assigning it to gen_loss.
        #       3) Backprop through the generator: update the gradients and optimizer.
        #### START CODE HERE ####
        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step()
        #### END CODE HERE ####

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 500: Generator loss: 0.6937802932262424, discriminator loss: 0.6956009175777437
Step 1000: Generator loss: 0.6942389067411421, discriminator loss: 0.6904130228757862
Step 1500: Generator loss: 0.6974336773157117, discriminator loss: 0.6881816349029537
Step 2000: Generator loss: 0.6987867375612256, discriminator loss: 0.6868392893075951
Step 2500: Generator loss: 0.697244793534279, discriminator loss: 0.687607634186745
Step 3000: Generator loss: 0.6939527454376218, discriminator loss: 0.6890402016639698
Step 3500: Generator loss: 0.6919104378223421, discriminator loss: 0.6910886815786361
Step 4000: Generator loss: 0.6926310302019107, discriminator loss: 0.6929280650615686
Step 4500: Generator loss: 0.6947240042686457, discriminator loss: 0.6932250317335127
Step 5000: Generator loss: 0.6947778234481807, discriminator loss: 0.6927064278125759
Step 5500: Generator loss: 0.6942368317842488, discriminator loss: 0.6926510924100868
Step 6000: Generator loss: 0.6949858790636061, discrimina

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 18000: Generator loss: 0.6931782385110858, discriminator loss: 0.6931095813512808
Step 18500: Generator loss: 0.69332435798645, discriminator loss: 0.6931282669305797
Step 19000: Generator loss: 0.6931697527170185, discriminator loss: 0.6930964077711106
Step 19500: Generator loss: 0.693090586662293, discriminator loss: 0.6932088235616682
Step 20000: Generator loss: 0.6931847213506706, discriminator loss: 0.6930512892007828
Step 20500: Generator loss: 0.6932008289098733, discriminator loss: 0.6931079108715057
Step 21000: Generator loss: 0.6931259059906013, discriminator loss: 0.6932020499706267
Step 21500: Generator loss: 0.6929983360767363, discriminator loss: 0.6932497819662097
Step 22000: Generator loss: 0.6930529710054396, discriminator loss: 0.6932317289113998
Step 22500: Generator loss: 0.693133000850678, discriminator loss: 0.6931378440856931
Step 23000: Generator loss: 0.6931802737712859, discriminator loss: 0.693065739750862
Step 23500: Generator loss: 0.6932120745182033, 

  0%|          | 0/17801 [00:00<?, ?it/s]

...

Step 125000: Generator loss: 0.6932632522583002, discriminator loss: 0.6931748557090761
Step 125500: Generator loss: 0.6928656479120252, discriminator loss: 0.6931262168884282
Step 126000: Generator loss: 0.6935334653854364, discriminator loss: 0.6931577869653701
Step 126500: Generator loss: 0.692934020519256, discriminator loss: 0.6931302894353871
Step 127000: Generator loss: 0.6932428342103947, discriminator loss: 0.6931304525136945
Step 127500: Generator loss: 0.6935575381517414, discriminator loss: 0.6931603447198872
Step 128000: Generator loss: 0.6934125179052353, discriminator loss: 0.6930666397809973
Step 128500: Generator loss: 0.6927049038410188, discriminator loss: 0.6932200343608851
Step 129000: Generator loss: 0.692866448402405, discriminator loss: 0.6932265551090244
Step 129500: Generator loss: 0.6930309489965432, discriminator loss: 0.6930230928659438
Step 130000: Generator loss: 0.6931900807619104, discriminator loss: 0.6931209154129035
Step 130500: Generator loss: 0.693

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 142500: Generator loss: 0.6933146018981937, discriminator loss: 0.6931725144386293
Step 143000: Generator loss: 0.6933557585477832, discriminator loss: 0.6931802872419367
Step 143500: Generator loss: 0.6932153031826016, discriminator loss: 0.6931383942365643
Step 144000: Generator loss: 0.6929997502565379, discriminator loss: 0.6930513316392902
Step 144500: Generator loss: 0.6930408828258506, discriminator loss: 0.6931836196184159
Step 145000: Generator loss: 0.6931558328866958, discriminator loss: 0.6931562303304676
Step 145500: Generator loss: 0.6930744310617453, discriminator loss: 0.693116318583489
Step 146000: Generator loss: 0.6930607420206072, discriminator loss: 0.6931698403358454
Step 146500: Generator loss: 0.6933952302932731, discriminator loss: 0.6931635723114016
Step 147000: Generator loss: 0.6930134036540989, discriminator loss: 0.693105811595917
Step 147500: Generator loss: 0.6929807766675944, discriminator loss: 0.693130001783371
Step 148000: Generator loss: 0.6932

  0%|          | 0/17801 [00:00<?, ?it/s]

Step 160500: Generator loss: 0.6925645201206216, discriminator loss: 0.6932294864654537
Step 161000: Generator loss: 0.6927841820716862, discriminator loss: 0.6930708869695665
Step 161500: Generator loss: 0.6930803084373472, discriminator loss: 0.6929909937381743
Step 162000: Generator loss: 0.692919366836548, discriminator loss: 0.6932393482923499
Step 162500: Generator loss: 0.693483819365501, discriminator loss: 0.6932316728830341
Step 163000: Generator loss: 0.6936026912927629, discriminator loss: 0.6931049360036854
Step 163500: Generator loss: 0.692899534106255, discriminator loss: 0.6931036133766175
Step 164000: Generator loss: 0.6932748746871946, discriminator loss: 0.6932038239240645
Step 164500: Generator loss: 0.6931298092603695, discriminator loss: 0.6932085360288617
Step 165000: Generator loss: 0.6934347605705264, discriminator loss: 0.6930510747432705
Step 165500: Generator loss: 0.6929430503845218, discriminator loss: 0.693115846157074
Step 166000: Generator loss: 0.69323

In [30]:
torch.save(gen, 'genmodel_2.pt')

In [31]:
torch.save(disc, 'discmodel_2.pt')

In [32]:
fake_noise = get_noise(cur_batch_size, z_dim, device=device)

In [33]:
fake = gen(fake_noise)

In [34]:
new_data=fake.detach().numpy()

In [35]:
new_df=pd.DataFrame(new_data)

In [36]:
minmaxscaler.inverse_transform(new_data)

array([[ 6.99537109e+04, -2.38821935e-03,  4.67908096e+00,
         2.52836204e+00, -7.13576198e-01,  1.90590787e+00,
        -1.47535744e+01,  6.51574707e+00, -7.68414259e+00,
        -4.05615282e+00,  1.49024025e-01, -1.94922876e+00,
         2.86578608e+00,  4.66719389e-01,  4.13640118e+00,
        -2.21028313e-01, -1.57623339e+00, -4.97476149e+00,
        -1.78648576e-01, -1.04865098e+00,  1.62485657e+01,
         5.09280825e+00,  4.77982235e+00,  4.82304192e+00,
        -3.93244445e-01,  1.00302577e+00, -6.45077288e-01,
         1.57242596e+00,  9.32176781e+00,  3.92154427e+01],
       [ 1.39731047e+05, -1.29017389e+00,  1.84246123e+00,
         2.72249132e-01, -4.37335491e-01,  7.10338306e+00,
        -6.18834352e+00,  3.88558807e+01, -2.73385835e+00,
        -2.17951107e+00,  4.18208694e+00,  1.91975963e+00,
         1.40272617e+00, -1.25648582e+00,  2.37768435e+00,
         1.86720550e-01, -3.49970460e-01, -3.92531347e+00,
         4.93980587e-01, -1.19621384e+00,  2.17213764e+

In [37]:
def create_new_data(n_times=5):
    data=[]
    for _ in range(n_times):
        fake_noise = get_noise(cur_batch_size, z_dim, device=device)
        fake = gen(fake_noise)
        new_data=fake.detach().numpy()
        data.append(minmaxscaler.inverse_transform(new_data))
    return pd.DataFrame(np.vstack(data))

create_new_data(n_times=3)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,114738.757812,-0.495996,0.325738,0.071241,4.110361,-10.919909,10.442595,-18.487663,9.639565,-4.181394,...,-15.237973,5.893458,-1.324298,3.071975,0.169218,-0.516669,-0.138654,0.024732,-8.608523,73.486557
1,96399.515625,-3.911854,8.167215,-1.107892,8.016469,-19.422447,17.240328,-16.974100,5.418760,-3.025454,...,-19.275404,0.561579,-0.915233,7.910861,-0.035102,1.339738,0.594294,-1.988063,-2.897775,202.804993
2,56187.730469,-1.431320,10.096639,-0.868973,-1.922716,11.985427,6.276679,-0.667818,-13.125577,-0.846630,...,12.990332,-4.363430,5.303897,-3.397654,1.447364,0.583149,-0.913898,-7.136986,11.239240,72.408485
3,144120.046875,-0.813232,0.745212,2.014179,-0.168681,2.168889,-13.471707,26.283175,-6.772091,-6.392673,...,20.970816,2.439697,4.788126,12.338380,0.484754,-0.974162,-0.796711,-11.433218,-3.341536,20.354683
4,84552.312500,-1.428741,9.166338,2.127709,1.902657,-3.239834,12.916766,-8.606132,-5.255139,-3.059400,...,2.937529,2.963536,0.053884,4.811018,0.603229,1.124938,0.549955,1.735524,-1.720056,162.852325
5,70316.609375,1.061002,3.285070,3.510750,-0.709053,8.961229,-12.310706,-9.315220,-4.258889,-2.695735,...,22.058533,-5.206933,2.004431,5.750990,0.607375,-1.756846,-0.390757,2.820733,-7.632590,49.209957
6,62327.832031,-0.628858,3.776641,0.031833,0.609544,-0.229271,-13.618052,6.784653,-6.628531,-3.567304,...,7.888371,0.930790,4.450389,-5.400101,-0.097051,1.289788,-0.208453,1.942836,9.643490,62.704918


In [41]:
df= pd.read_csv('../credit_card_kaggle/creditcard.csv')
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [42]:
df.shape

(284807, 31)

In [40]:
syn_data = create_new_data(n_times=300)
syn_data.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,...,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000
mean,90777.179688,-0.955830,4.904205,1.029115,1.495160,-2.132203,1.322069,-2.726856,-3.246976,-3.432107,...,5.445347,0.272328,2.135392,3.774948,0.457344,0.175984,-0.196586,-2.359477,-0.386688,85.639893
std,28118.839844,1.132654,5.102355,1.974553,2.787741,12.167484,14.033566,16.310694,6.586971,1.643862,...,14.242802,4.902923,2.351805,5.252159,0.553893,1.477281,0.483806,4.138632,6.827394,43.368999
min,31586.849609,-5.815773,-10.813290,-6.231070,-2.977193,-51.078560,-21.252434,-36.173141,-20.713285,-8.755115,...,-33.179539,-16.335352,-5.138299,-11.660846,-0.922489,-3.931865,-1.342366,-15.707978,-10.492925,12.630581
25%,70007.949219,-1.597516,1.113627,-0.256436,-0.601841,-7.547033,-11.007743,-12.896372,-7.672051,-4.562226,...,-2.639644,-2.978846,0.688233,0.198825,0.055503,-0.809952,-0.548776,-4.802675,-6.044104,55.223634
50%,90142.933594,-0.775554,4.633291,1.109696,0.738410,0.121257,-0.531104,-3.650452,-3.928678,-3.454251,...,9.265893,0.603183,2.631333,4.959139,0.420844,0.208589,-0.294161,-1.905589,-1.962419,78.285992
75%,113983.943359,-0.143181,8.587939,2.431238,2.889458,6.261565,14.085660,5.121525,0.090044,-2.268887,...,16.813184,3.900380,3.938228,7.890928,0.795609,1.182075,0.082618,0.535096,4.177539,109.044353
max,153830.468750,1.449201,17.591249,6.063201,10.310723,22.234766,32.328087,59.128685,14.748790,1.841690,...,28.069431,12.828651,6.740103,13.914111,2.567192,4.182652,1.293710,9.079445,18.672047,376.584747
